In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import tensorflow_hub as hub

In [2]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=[
   tfds.Split.TRAIN.subsplit(tfds.percent[:60]),
   tfds.Split.TRAIN.subsplit(tfds.percent[60:]),
   tfds.Split.TEST
],
    as_supervised=True)

In [3]:
train_data.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [41]:
embedding = "nnlm-en-dim50_2/"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [46]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()


Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_10 (KerasLayer)  (None, 50)                48190600  
_________________________________________________________________
dense_5 (Dense)              (None, 64)                3264      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 48,198,089
Trainable params: 48,198,089
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [49]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 26s 852ms/step - loss: 0.6392 - accuracy: 0.5701 - val_loss: 0.5524 - val_accuracy: 0.6702
Epoch 2/10
30/30 [==============================] - 19s 642ms/step - loss: 0.4606 - accuracy: 0.7717 - val_loss: 0.3778 - val_accuracy: 0.8227
Epoch 3/10
30/30 [==============================] - 19s 648ms/step - loss: 0.2946 - accuracy: 0.8753 - val_loss: 0.3306 - val_accuracy: 0.8601
Epoch 4/10
30/30 [==============================] - 19s 635ms/step - loss: 0.1886 - accuracy: 0.9291 - val_loss: 0.3276 - val_accuracy: 0.8663
Epoch 5/10
30/30 [==============================] - 20s 652ms/step - loss: 0.1131 - accuracy: 0.9622 - val_loss: 0.3508 - val_accuracy: 0.8674
Epoch 6/10
30/30 [==============================] - 19s 650ms/step - loss: 0.0655 - accuracy: 0.9807 - val_loss: 0.3921 - val_accuracy: 0.8685
Epoch 7/10
30/30 [==============================] - 20s 659ms/step - loss: 0.0341 - accuracy: 0.9915 - val_loss: 0.4618 - val_accuracy: 0.8641

In [56]:
model.save('model_py/model.h5')